In [2]:
import os
import re
import pandas as pd

path = [
    r'C:\Users\Ryu\Python_project\data\battery251027\Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc'
]

In [3]:
# 충방전기 구분 (패턴 폴더 유무로 구분)
def check_cycler(raw_file_path): 
    # 충방전기 데이터 폴더에 패턴 폴더 유무로 PNE와 Toyo 구분, PNE이면 True, Toyo이면 False
    cycler = os.path.isdir(raw_file_path + "\\Pattern")
    return cycler

# filepath 이름에서 용량을 추출하는 함수
def name_capacity(data_file_path):
    # 원시 문자열을 사용하여 특수 문자를 공백으로 대체
    raw_file_path = re.sub(r'[._@\$$$$$$\(\)]', ' ', data_file_path)
    # 정규 표현식을 사용하여 "mAh"로 끝나는 용량 값을 찾습니다. (소수점 포함)
    match = re.search(r'(\d+([\-.]\d+)?)mAh', raw_file_path)
    # 소수점 용량을 위해 -를 .으로 변환
    min_cap = match.group(1).replace('-', '.')
    # 일치하는 값이 있으면 실수로 변환하여 반환
    if min_cap:
        return float(min_cap)
    # 용량 값이 없으면 None을 반환하거나 오류를 발생시킵니다.
    return None

In [4]:
if(not check_cycler(path[0])):
    print("Toyo 충방전기 데이터입니다.")
    Rated_capacity = name_capacity(path[0])   

Toyo 충방전기 데이터입니다.


In [ ]:
def toyo_read_csv(file_path, read_all_files=False):
    """
    Toyo 충방전기 데이터를 읽는 함수.
    read_all_files가 False이면 capacity.log를, True이면 6자리 숫자 파일을 모두 읽어 합칩니다.
    만약 6자리 숫자 파일이 현재 경로에 없으면, 숫자 이름의 하위 폴더에서 6자리 숫자 파일을 찾아 합칩니다.
    """
    # capacity.log 파일을 읽는 경우
    if not read_all_files:
        filepath = os.path.join(file_path, 'capacity.log')
        if os.path.isfile(filepath):
            return pd.read_csv(filepath, sep=",", skiprows=0, engine="c", encoding="cp949", on_bad_lines='skip')
        else:
            print(f"File not found: {filepath}")
            return pd.DataFrame()
            
    # 6자리 숫자 형식의 모든 파일을 읽는 경우
    else:
        df_list = []
        try:
            # 1. 현재 경로에서 6자리 숫자 파일 검색
            cycle_files = sorted([f for f in os.listdir(file_path) if re.match(r'^\d{6}$', f) and os.path.isfile(os.path.join(file_path, f))])
            
            if cycle_files:
                for filename in cycle_files:
                    filepath = os.path.join(file_path, filename)
                    temp_df = pd.read_csv(filepath, sep=",", skiprows=3, engine="c", encoding="cp949", on_bad_lines='skip')
                    df_list.append(temp_df)
            else:
                # 2. 현재 경로에 파일이 없으면 숫자 이름의 하위 폴더 검색
                subdirs = sorted([d for d in os.listdir(file_path) if re.match(r'^\d+$', d) and os.path.isdir(os.path.join(file_path, d))], key=int)

                if not subdirs:
                    print(f"No 6-digit files or numeric subdirectories found in {file_path}")
                    return pd.DataFrame()

                for subdir in subdirs:
                    subdir_path = os.path.join(file_path, subdir)
                    cycle_files_in_subdir = sorted([f for f in os.listdir(subdir_path) if re.match(r'^\d{6}$', f)])
                    
                    for filename in cycle_files_in_subdir:
                        filepath = os.path.join(subdir_path, filename)
                        temp_df = pd.read_csv(filepath, sep=",", skiprows=3, engine="c", encoding="cp949", on_bad_lines='skip')
                        df_list.append(temp_df)

            # 모든 데이터프레임을 하나로 합칩니다.
            if df_list:
                return pd.concat(df_list, ignore_index=True)
            else:
                return pd.DataFrame()

        except FileNotFoundError:
            print(f"Directory not found: {file_path}")
            return pd.DataFrame()
        except Exception as e:
            print(f"An error occurred: {e}")
            return pd.DataFrame()
        

df = pd.DataFrame()
# 6자리 숫자 파일을 모두 읽으려면 read_all_files=True 옵션을 사용합니다.
df.raw = toyo_read_csv(path[0], read_all_files=True)
df.raw.head()

No 6-digit numeric files found in C:\Users\Ryu\Python_project\data\battery251027\Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc


C:\Users\Ryu\AppData\Local\Temp\ipykernel_21536\1179717089.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.raw = toyo_read_csv(path[0], read_all_files=True)


""
